In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns 

from matplotlib import pyplot as plt

In [4]:
sirio_libanes = pd.read_excel('https://github.com/ConradBitt/BootCamp_DataScience/blob/master/ML%20em%20Saude/dados/Kaggle_Sirio_Libanes_ICU_Prediction.xlsx?raw=true')

In [41]:
aih_sp = pd.read_csv('https://raw.githubusercontent.com/ConradBitt/BootCamp_DataScience/master/projeto_final/dados/AIH_aprovadas_por_AnoMes_atendimento_segundo_Munic%C3%ADpio_SP.csv',
                     sep=';', skiprows=4, skipfooter=13, engine='python', encoding='iso-8859-1').set_index('Município')
aih_sp.index = aih_sp.index = ['sao_paulo']
aih_sp = aih_sp.iloc[:,:-1]

In [53]:
leitos_internacoes = pd.read_csv('https://raw.githubusercontent.com/ConradBitt/BootCamp_DataScience/master/projeto_final/dados/Leitos_e_Internacoes.csv',
                                 sep=';', decimal=',')

In [55]:
leitos_internacoes.query('nome_drs == "DRS 01 Grande São Paulo"')

,datahora,nome_drs,pacientes_uti_mm7d,total_covid_uti_mm7d,ocupacao_leitos,pop,leitos_pc,internacoes_7d,internacoes_7d_l,internacoes_7v7
0,08/10/2020,DRS 01 Grande São Paulo,2036.86,4829.86,42.17,21138247.0,22.85,4121.0,4093.0,0.68
18,09/10/2020,DRS 01 Grande São Paulo,2027.43,4822.29,42.04,21138247.0,22.81,4146.0,4130.0,0.39
36,10/10/2020,DRS 01 Grande São Paulo,2022.00,4812.57,42.01,21138247.0,22.77,4164.0,4111.0,1.29
54,11/10/2020,DRS 01 Grande São Paulo,2007.00,4781.86,41.97,21138247.0,22.62,4110.0,4084.0,0.64
72,12/10/2020,DRS 01 Grande São Paulo,1990.29,4748.71,41.91,21138247.0,22.47,4034.0,4106.0,-1.75
...,...,...,...,...,...,...,...,...,...,...
2394,18/02/2021,DRS 01 Grande São Paulo,3165.57,4794.43,66.03,21138247.0,22.68,5067.0,4788.0,5.83
2412,19/02/2021,DRS 01 Grande São Paulo,3203.86,4793.29,66.84,21138247.0,22.68,5202.0,4745.0,9.63
2430,20/02/2021,DRS 01 Grande São Paulo,3243.00,4818.71,67.30,21138247.0,22.80,5345.0,4782.0,11.77
2448,21/02/2021,DRS 01 Grande São Paulo,3278.57,4839.86,67.74,21138247.0,22.90,5489.0,4826.0,13.74
